In [1]:
!git clone https://github.com/dovugiacan14/Image-Classification-CIFAR100

Cloning into 'Image-Classification-CIFAR100'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 119 (delta 65), reused 86 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (119/119), 26.36 KiB | 5.27 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [2]:
!pip install torch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.3 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitli

In [3]:
%%writefile /kaggle/working/Image-Classification-CIFAR100/config.py
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch_optimizer import Lookahead

device = "cuda" if torch.cuda.is_available() else "cpu"
save_dir = "checkpoints/"
os.makedirs(save_dir, exist_ok=True)
num_workers = 16


class CNNConfig:
    criterion = nn.CrossEntropyLoss()
    learning_rate = 0.001
    num_epochs = 100
    batch_size = 8
    device = device
    out_name = "cnn_model"

    @staticmethod
    def optimizer_fn(model):
        return optim.Adam(model.parameters(), lr=CNNConfig.learning_rate)


class ResNetConfig:
    criterion = nn.CrossEntropyLoss()
    learning_rate = 1e-4
    weight_decay = 1e-4
    num_epochs = 100
    batch_size = 32
    device = device
    out_name = save_dir + "ResNet50/resnet50_model"
    num_workers = num_workers

    @staticmethod
    def optimizer_fn(model):
        return optim.AdamW(
            model.parameters(),
            lr=ResNetConfig.learning_rate,
            weight_decay=ResNetConfig.weight_decay,
        )

    @staticmethod
    def scheduler_fn(optimizer):
        return optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=ResNetConfig.num_epochs
        )


class VGGConfig:
    criterion = nn.CrossEntropyLoss()
    batch_size = 8
    num_epochs = 100
    learning_rate = 1e-4
    device = device
    out_name = "vgg16_model"

    @staticmethod
    def optimizer_fn(model):
        return optim.Adam(model.parameters(), lr=VGGConfig.learning_rate)


class DenseNetConfig:
    criterion = nn.CrossEntropyLoss()
    learning_rate = 1e-4
    weight_decay = 1e-4
    num_epochs = 100
    batch_size = 32
    device = device
    out_name = save_dir + "DenseNet/densenet121_model"

    @staticmethod
    def optimizer_fn(model):
        return optim.AdamW(
            model.parameters(),
            lr=DenseNetConfig.learning_rate,
            weight_decay=DenseNetConfig.weight_decay,
        )


class EfficientConfig:
    criterion = nn.CrossEntropyLoss()
    batch_size = 32
    num_epochs = 2
    learning_rate = 1e-4
    device = device
    out_name = "efficientnetb0_finetune_model"

    @staticmethod
    def optimizer_fn(model):
        return optim.Adam(model.parameters(), lr=EfficientConfig.learning_rate)


class ConvNeXtConfig:
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    learning_rate = 3e-4
    num_epochs = 100
    batch_size = 32
    device = device
    out_name = save_dir + "ConvNeXt/convnext_tiny_model"
    grad_clip = 1.0  # gradient clipping value

    @staticmethod
    def optimizer_fn(model):
        base_opt = optim.AdamW(
            model.parameters(), lr=ConvNeXtConfig.learning_rate, weight_decay=1e-4
        )
        return Lookahead(base_opt)


class ViTConfig:
    criterion = nn.CrossEntropyLoss()
    num_epochs = 2
    batch_size = 32
    learning_rate = 2e-5
    weight_decay = 0.01
    T_max = 10
    device = device
    out_name = "ViT_finetune_model"

    @staticmethod
    def optimizer_fn(model):
        return optim.AdamW(model.parameters(), lr=ViTConfig.learning_rate)

    @staticmethod
    def scheduler():
        return optim.lr_scheduler.CosineAnnealingLR(
            optimizer=ViTConfig.optimizer_fn, T_max=ViTConfig.T_max
        )


class SwinConfig:
    criterion = nn.CrossEntropyLoss()
    learning_rate = 2e-5
    weight_decay = 0.01
    num_epochs = 100
    batch_size = 32
    device = device
    out_name = save_dir + "SwinTransformer/swin_transformer_tiny_model"
    model_name = "microsoft/swin-tiny-patch4-window7-224"

    @staticmethod
    def optimizer_fn(model):
        return optim.AdamW(
            model.parameters(),
            lr=SwinConfig.learning_rate,
            weight_decay=SwinConfig.weight_decay,
        )

Overwriting /kaggle/working/Image-Classification-CIFAR100/config.py


In [ ]:
!python /kaggle/working/Image-Classification-CIFAR100/main.py --option 3

2025-05-31 12:57:26.115162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748696246.297817      99 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748696246.352391      99 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|████████████████████████████████████████| 169M/169M [00:12<00:00, 13.7MB/s]
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
            Conv2d-2         [-1, 64, 224, 224]          36,928
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 12